# **Generating CBR DataFrame**

In [2]:
import sys
import os
import pandas as pd
from natsort import natsorted
import decimal
pd.set_option('display.max_rows', None)

**Variables to Set**

In [8]:
directory = 'HIghway3km'
cbr_file_name = '0cbrqtest.txt'

##### **Viewing Transmission Powers**

In [9]:
arr = natsorted(os.listdir(directory))
arr

['2mW', '4mW', '6mW', '8mW', '10mW', '12mW', '14mW', '16mW', '18mW', '20mW']

##### **Generating CBR Values and Appending Data to Lists**

In [25]:
trans_power = []
vehicle_density = []
cbr_str = []
 
for filename_1 in natsorted(os.listdir(directory)):
    f = os.path.join(directory, filename_1)
    for filename_2 in os.listdir(f):
        b = os.path.join(f, filename_2) 
        g = os.path.join(b, cbr_file_name)
        file = open(g, "r")
        data = file.readline().strip('\n')
        sumCBR = 0
        numVals = 0
        j = 0
        curIntervalStart = 0
        cbrStr = ""
        while data:
            first = data.find('\t')
            second = data.find('\t', first+1)
            third = data.find('\t', second+1)
            simTime = eval(data[first+1:second])
            cbr = eval(data[third+1:])
            if simTime <= curIntervalStart + 5:
                if cbr != -1:
                    sumCBR += cbr
                    numVals += 1
            else:
                if numVals > 0:
                    cbrStr += str(curIntervalStart) + " - " + str(curIntervalStart + 5) + '\t' + str(sumCBR/numVals) + '\n'
                    sumCBR = 0
                    numVals = 0
                    curIntervalStart += 5
            j+=1
            if j%10000 == 0:
                print(j)
            data = file.readline().strip('\n')
        cbrStr += str(curIntervalStart) + " - " + str(curIntervalStart + 5) + \
                      '\t' + str(sumCBR/numVals) + '\n'
       
        trans_power.append(filename_1)
        vehicle_density.append(filename_2)
        #val = sumCBR/numVals
        val = round(decimal.Decimal(sumCBR) / decimal.Decimal(numVals), 9)
        cbr_str.append(val)      

**Creating DataFrame**

In [3]:
df = pd.DataFrame(list(zip(trans_power, vehicle_density, cbr_str)),
               columns =['Transmission Power', 'Vehicle Density', 'CBR'], index=None)
df

NameError: name 'trans_power' is not defined

**Outputting CSV File**

In [6]:
df.to_csv("cbr_results", index=False)